Import libraries

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from statsmodels.tsa.stattools import coint
import seaborn as sns
import winsound
import math
from statistics import mean

In [29]:
df = pd.read_csv("Prices200.csv")
df.drop(["Name"],axis=1,inplace=True)

In [30]:
df['date']= pd.to_datetime(df['date'],dayfirst=True)

Remove all tickers other than stock1 and stock2

In [31]:
df1=df
stock1='GAIL IS Equity' 
stock2='RIL IS Equity'
indices=[]
for i in range(len(df1)):
    if i%10000==0:
        print(i)
    if df1.iloc[i]['Ticker']!=stock1 and df1.iloc[i]['Ticker']!=stock2:
        indices.append(i)
df1.drop(df1.index[indices],inplace=True)
#print(len(df1))
winsound.Beep(500,1000)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000


Keep only those values which fall in selected financial year

In [32]:
#df1=df
start=datetime.datetime(2018, 4, 2)
end=datetime.datetime(2019, 3, 31)
indices=[]
for i in range(len(df1)):
    if i%100==0:
        print(i)
    if df1.iloc[i]['date']<start or df1.iloc[i]['date']>end:
        indices.append(i)
df1.drop(df1.index[indices],inplace=True)
#print(len(df1))
#winsound.Beep(500,1000)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


Keep only values from last 6 months of financial year where we are actually going to trade

In [33]:
date=[]
midpt = datetime.datetime(2018, 9, 27)
for i in range(len(df1)):
    if df1.iloc[i]['date']<=midpt:
        date.append(i)
df1.drop(df1.index[date],inplace=True)
#print(len(df1))

Make lists of stock1 price, stock2 price and trading dates. Will append these to another dataframe.

In [34]:
#df1.head()
stock1_price=[]
stock2_price=[]
trading_dates=[]
for i in range(len(df1)):
    #if i%10000==0:
        #print(i)
    if df1.iloc[i]['Ticker']==stock1:
        stock1_price.append(df1.iloc[i]['PX_LAST'])
        trading_dates.append(df1.iloc[i]['date'])
    elif df1.iloc[i]['Ticker']==stock2:
        stock2_price.append(df1.iloc[i]['PX_LAST'])

In [35]:
#print(len(stock1_price))
#print(len(stock2_price))
#print(len(trading_dates))

Make new dataframe

In [36]:
df2=pd.DataFrame()
df2['dates']=trading_dates
df2[stock1]=stock1_price
df2[stock2]=stock2_price
df2.head()

,dates,GAIL IS Equity,RIL IS Equity
0,2018-09-28,189.50,1257.95
1,2018-10-01,190.50,1231.70
2,2018-10-03,190.88,1205.10
3,2018-10-04,183.98,1122.25
4,2018-10-05,166.03,1048.85


Code to export dataframe into csv file

In [93]:
df2.to_csv(r'C:\Users\anupi\Desktop\ISB-CAF\Aniket_Bloomberg\Aniket_Bloomberg\rilgail.csv')

Take ln of ratio of stock prices, form a list and add it to df2

In [38]:
ln_ratio = []
for i in range(len(df2)):
    ln_ratio.append(math.log(df2.iloc[i][stock2]/df2.iloc[i][stock1]))
df2['ln']=ln_ratio
#df2.head()

Code to make list of ratios and add it to df2 (not needed in normal scenario)

In [39]:
#ratio = []
#for i in range(len(df2)):
#    ratio.append(df2.iloc[i][stock2]/df2.iloc[i][stock1])
#df2['ratio']=ratio

Add moving average of last 10 days of ln(ratio), std dev and z score

In [40]:
ln_mvg_avg=[]
#mvg_avg=[]
ln_std_dev=[]
#std_dev=[]
for i in range(len(df2)):
    last10=[]
    ln_last10=[]
    if i<=9:
        ln_mvg_avg.append(0)
        #mvg_avg.append(0)
        ln_std_dev.append(0)
        #std_dev.append(0)
    else:
        for j in range(0,10):
            ln_last10.append(df2.iloc[i-j-1]['ln'])
            #last10.append(df2.iloc[i-j-1]['ratio'])
        ln_mvg_avg.append(mean(ln_last10))
        #mvg_avg.append(mean(last10))
        ln_std_dev.append(np.std(ln_last10))
        #std_dev.append(np.std(last10))

df2['ln_mvg_avg']=ln_mvg_avg
df2['ln_std_dev']=ln_std_dev
#df2['mvg_avg']=mvg_avg
#df2['std_dev']=std_dev

ln_z_score=[]
#z_score=[]
for i in range(len(df2)):
    if i<=9:
        ln_z_score.append(0)
#        z_score.append(0)
    else:
        ln_z_score.append((df2.iloc[i]['ln']-df2.iloc[i]['ln_mvg_avg'])/df2.iloc[i]['ln_std_dev'])
#        z_score.append((df2.iloc[i]['ratio']-df2.iloc[i]['mvg_avg'])/df2.iloc[i]['std_dev'])
df2['ln_z_score']=ln_z_score
#df2['z_score']=z_score
#df2.head(15)

Select buy_threshold and sell_threshold manually, 2 means 2 SD

In [41]:
buy_threshold=-2
sell_threshold=2

Calculate signal based on whether the z_score crosses the thresholds defined earlier

In [42]:
ln_signal=[]
#signal=[]

for i in range(len(df2)):
    if (df2.iloc[i]['ln_z_score']<buy_threshold):
        ln_signal.append('BUY')
    elif (df2.iloc[i]['ln_z_score']>sell_threshold):
        ln_signal.append('SELL')
    else:
        ln_signal.append('')
df2['ln_signal']=ln_signal

#for i in range(len(df2)):
    #if (df2.iloc[i]['z_score']<buy_threshold):
        #signal.append('BUY')
    #elif (df2.iloc[i]['z_score']>sell_threshold):
        #signal.append('SELL')
    #else:
        #signal.append('')
#df2['signal']=signal



#df2.head(25)

Empty list formation for the other columns which need to be added to df2 

In [132]:
status=['']*124
buy_price=[0]*124
sell_price=[0]*124
mtm=[0]*124
SL = -100
TP = 200
#stock1_size=5
#stock2_size=3

In [133]:
price_ratio = df2.iloc[0][stock2]/df2.iloc[0][stock1]
price_ratio = round(price_ratio)
print(price_ratio)
#print(round(price_ratio))

7.0


In [134]:
stock1_size=price_ratio
stock2_size=1

Code to populate the mtm, status, buy_price and sell_price. Order very important.

In [135]:
for i in range(len(df2)):
    #print (i)

        
#code for mtm
    if i==0:
        continue
    else:
        if status[i-1]=='BUY':
            #print (i)
            #break
            mtm[i]=(sell_price[i-1]-df2.iloc[i][stock1])*stock1_size+(df2.iloc[i][stock2]-buy_price[i-1])*stock2_size
        elif status[i-1]=='SELL':
            mtm[i]=(sell_price[i-1]-df2.iloc[i][stock2])*stock2_size+(df2.iloc[i][stock1]-buy_price[i-1])*stock1_size
        else:
            mtm[i]=0
    
#code for status
    
    
    if status[i-1]=='' or status[i-1]=='SL' or status[i-1]=='TP':
        status[i]= df2.iloc[i]['ln_signal']
    elif mtm[i]==0:
        status[i]=mtm[i]
    elif mtm[i]< SL:
        status[i]= 'SL'
    elif mtm[i]>TP:
        status[i]= 'TP'
    else:
        status[i]=status[i-1]

#code for buy price
    if status[i]==status[i-1]:
        buy_price[i]=buy_price[i-1]
    elif status[i]=='SL' or status[i]=='TP':
        buy_price[i]=0
    elif df2.iloc[i]['ln_signal']=='BUY':
        buy_price[i]=df2.iloc[i][stock2]
    elif df2.iloc[i]['ln_signal']=='SELL':
        buy_price[i]=df2.iloc[i][stock1]
    else:
        buy_price[i]=0
    #print (type(buy_price))
    
#code for sell price
    if status[i]==status[i-1]:
        sell_price[i]=sell_price[i-1]
    elif status[i]=='SL' or status[i]=='TP':
        sell_price[i]=0
    elif df2.iloc[i]['ln_signal']=='BUY':
        sell_price[i]=df2.iloc[i][stock1]
    elif df2.iloc[i]['ln_signal']=='SELL':
        sell_price[i]=df2.iloc[i][stock2]
    else:
        sell_price[i]=0

Add the obtained lists to df2

In [136]:
df2['buy_price']=buy_price
df2['sell_price']=sell_price
df2['mtm']=mtm
df2['status']=status

In [137]:
df2.head(25)

,dates,GAIL IS Equity,RIL IS Equity,ln,ln_mvg_avg,ln_std_dev,ln_z_score,ln_signal,buy_price,sell_price,mtm,status,pnl,cumulative_pnl
0,2018-09-28,189.50,1257.95,1.892850,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
1,2018-10-01,190.50,1231.70,1.866498,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
2,2018-10-03,190.88,1205.10,1.842673,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
3,2018-10-04,183.98,1122.25,1.808264,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
4,2018-10-05,166.03,1048.85,1.843281,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
5,2018-10-08,171.95,1109.40,1.864371,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
6,2018-10-09,166.75,1090.05,1.877483,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
7,2018-10-10,168.18,1102.10,1.879938,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
8,2018-10-11,175.13,1087.80,1.826384,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0
9,2018-10-12,179.65,1126.55,1.835905,0.000000,0.000000,0.000000,,0.0,0.00,0.00,,0.0,0.0


In [138]:
pnl=[0]*124
cumulative_pnl=[0]*124

for i in range (len(df2)):
    if status[i]=='TP' or status[i]=='SL':
        pnl[i]=df2.iloc[i]['mtm']
    else:
        pnl[i]=0
    if i ==0:
        continue
    else:
        cumulative_pnl[i]=cumulative_pnl[i-1]+pnl[i]

df2['pnl']=pnl
df2['cumulative_pnl']=cumulative_pnl
        

In [139]:
brokerage = 0.0001

In [140]:
loss_SL = 0
profit_TP = 0
total_pnl= 0
count_SL= 0
count_TP=0
#avg_pnl= 0
tran_cost=0
#fund_used=0
per_returns=0
for i in range (len(df2)):
    if df2.iloc[i]['status']=='SL':
        loss_SL=loss_SL+df2.iloc[i]['mtm']
        count_SL=count_SL+1
    if df2.iloc[i]['status']=='TP':
        profit_TP=profit_TP+df2.iloc[i]['mtm']
        count_TP=count_TP+1
    if df2.iloc[i]['ln_signal']=='BUY' or df2.iloc[i]['ln_signal']=='SELL':
        tran_cost=tran_cost+ (stock1_size*df2.iloc[i][stock1]+stock2_size*df2.iloc[i][stock2])*brokerage
    if df2.iloc[i]['status']=='BUY' and df2.iloc[i-1]['status']!='BUY':
        for j in range (i+1,len(df2)):
            if df2.iloc[j]['status']!='BUY':
                per_returns = per_returns+((df2.iloc[j][stock2]-df2.iloc[i][stock2])*100)/df2.iloc[i][stock2]+((df2.iloc[i][stock1]-df2.iloc[j][stock1])*100)/df2.iloc[j][stock1]
    elif df2.iloc[i]['status']=='SELL' and df2.iloc[i-1]['status']!='SELL':
        for j in range (i+1,len(df2)):
            if df2.iloc[j]['status']!='BUY':
                per_returns = per_returns+((df2.iloc[j][stock1]-df2.iloc[i][stock1])*100)/df2.iloc[i][stock1]+((df2.iloc[i][stock2]-df2.iloc[j][stock2])*100)/df2.iloc[j][stock2]

                
                
                

                
                
#        fund_used=fund_used+(stock2_size*df2.iloc[i][stock2]+stock1_size*df2.iloc[i][stock1])
 #   elif df2.iloc[i]['status']=='SELL' and df2.iloc[i-1]['status']!='SELL':
  #      fund_used=fund_used+(stock1_size*df2.iloc[i][stock1]+stock2_size*df2.iloc[i][stock2])
        

total_pnl=profit_TP+loss_SL
net_pnl=total_pnl-tran_cost
#avg_pnl = total_pnl/(count_SL+count_TP)
#net_avg_pnl = net_pnl/(count_SL+count_TP)
#returns_6mon= (net_pnl*2)/1000

#print("Funds used = ", abs(fund_used))
print("Total PnL = ", total_pnl)
print("Net PnL (includes transaction costs) = ", net_pnl)
print("Percentage return = ", per_returns)
#print("Average PnL (PnL per trade) = ", avg_pnl)
#print("Net Average PnL (includes transaction costs) = ", total_pnl)
#print("Returns over 6 months (%)= ", returns_6mon, "%")


        

Total PnL =  200.56000000000006
Net PnL (includes transaction costs) =  195.56659800000006
Percentage return =  1273.4589074328292


In [141]:
#winsound.PlaySound('your-code-run-has-completed.wav', winsound.SND_FILENAME)